In [1]:
import random
import math

In [2]:
def shuffleUsers(nbUsers):
    
    users = [int(i) for i in range(nbUsers)]
    
    random.shuffle(users)
    
    return users

In [3]:
def solutionInTabu(T, newSolution):
    for l in T:
        if newSolution == l:
            return True
    return False

In [4]:
def isFeasible(nbResources, solution):
    for j in range(nbResources):
        if solution[j]:
            return True
    return False

In [5]:
def initialize(nbResources, probability):
    solution = []
    for j in range(nbResources):
        solution.append(random.random() < probability)
        
    if not isFeasible(nbResources, solution):
        solution[random.randrange(nbResources)] = True
        
    return solution

In [6]:
def invert(nbResources, solution):
    newSolution = list(solution)
    j = random.randrange(nbResources)
    newSolution[j] = not newSolution[j]
    if isFeasible(nbResources, newSolution):
        return j, newSolution
    return -1, solution

In [7]:
def restore(j, sol):
    sol[j] = not sol[j]

def restoreStorages(changes):
    for i in changes:
        storages[changes[i]] += demands[i]

In [8]:
def readInput(filename):
    input = open(filename, "r")    
    nbResources, nbUsers = [int(i) for i in input.readline().split()]
    storages = []
    fixedCost = []
    demands = []
    cost = []
    for i in range(nbResources):
        x, y = [int(i) for i in input.readline().split()]
        storages.append(x)
        fixedCost.append(y)
    for i in range(nbUsers):
        x, _ = [int(i) for i in input.readline().split()]
        demands.append(x)
        cost.append([float(j) for j in input.readline().split()])
    
    return nbUsers, nbResources, cost, fixedCost, demands, storages

In [9]:
def solutionValue(nbUsers, nbResources, cost, fixedCost,\
                  solution, demands, storages):
    globalValue = float('inf')
    value = 0.0
    used = [False] * nbResources
    changes = {}
    
    for i in range(math.ceil(math.sqrt(nbUsers))):
        users = shuffleUsers(nbUsers)
    
        for i in users:
            minCost = float('inf')
            jUsed = -1
            for j in range(nbResources):
                if (solution[j] and cost[i][j] < minCost) and \
                        (demands[i] <= storages[j]):
                    minCost = cost[i][j]
                    jUsed = j

            if (jUsed == -1):
                restoreStorages(changes)
                return False

            value += minCost
            storages[jUsed] -= demands[i]
            used[jUsed] = True

            changes[i] = jUsed

        for j in range(nbResources):
            if used[j]:
                value += fixedCost[j]
                
        
        solution = used
        restoreStorages(changes)
        
        if value < globalValue:
            globalValue = value
    
    return globalValue

In [10]:
def tabuSearch(nbUsers, nbResources, cost, fixedCost,\
               demands, storages, solution, maxIters, tabuListLength):
    
    for i in range(maxIters):
        currValue = solutionValue(nbUsers, nbResources, cost, fixedCost, solution, demands, storages)
        if(not currValue):
            continue
        break
    
    if (currValue == False): # i == maxIters
        print('Starting point not found!\n')
        return False
    
    bestValue = currValue
   
    i = 0
    T = []
    
    T.append(solution)
    
    while (i < maxIters):
        
        j, newSolution = invert(nbResources, solution)
        solution = newSolution
        
        if j < 0:
            continue
        
        if (solutionInTabu(T, newSolution)):
            i+=1
            continue
            
        newValue = solutionValue(nbUsers, nbResources, cost, fixedCost, newSolution, demands, storages)

        
        if (newValue < currValue and newValue):
            currValue = newValue
        else:
            restore(j, newSolution)
            
        if (newValue < bestValue and newValue):
            bestValue = newValue
            if(len(T) >= tabuListLength):
                del T[0]
            T.append(newSolution)
        i += 1
        
    return bestValue

In [11]:
nbUsers, nbResources, cost, fixedCost, demands, storages = readInput('testExamples/cap51')
#nbUsers = 10
#nbResources = 10
#print(nbUsers)
#print(nbResources)
#print(shuffleUsers(nbUsers))
#print('Fixed cost\\n', fixedCost)
#print('Demands\\n', demands)
#print('Cost\\n{}'.format(cost))
#print('Storages\\n', storages)
#print(demands)
#print(sum(demands))
prime = float('inf')

for i in range(100):
    solution = initialize(nbResources, 0.7)
    bestValue = tabuSearch(nbUsers, nbResources, cost , fixedCost, demands, storages, solution, 1000, 100)

    if ((bestValue < prime) & (bestValue != False)):
        prime = bestValue
        
print('Best value:', prime)

Best value: 1012186.9749999997
